In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, Point, mapping
from shapely.ops import transform
import folium
from pyproj import Transformer
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:

citygml_path = '/home/elsherif/Desktop/Thesis/Lod2/Neuberg/662_5400.gml'
geojson_path = '/home/elsherif/Segmentation Project/pygeo/need-api-skeleton/data/rooftop_blobs2024.geojson'
out_matches_csv = 'roof_matches.csv'
out_matches_geojson = 'roof_matches.geojson'

In [3]:

# --------- Cell 3: Parse CityGML and extract RoofSurface polygons (2D) ---------
# This function extracts all RoofSurface posList polygons (drops Z)
def extract_roof_polygons_from_gml(gml_path):
    tree = ET.parse(gml_path)
    root = tree.getroot()
    namespaces = dict(node for _, node in ET.iterparse(gml_path, events=['start-ns']))
    # find building namespace
    bldg_ns = None
    for k, v in namespaces.items():
        if 'building' in v.lower():
            bldg_ns = v
            break
    gml_ns = namespaces.get('gml', 'http://www.opengis.net/gml')

    roof_polys = []
    roof_meta = []
    for bi, building in enumerate(root.findall(f".//{{{bldg_ns}}}Building")):
        b_id = building.attrib.get('{http://www.opengis.net/gml}id', f'building_{bi}')
        for bounded in building.findall(f".//{{{bldg_ns}}}boundedBy"):
            surface = list(bounded)[0] if list(bounded) else None
            if surface is None:
                continue
            surface_type = surface.tag.split('}')[-1]
            if surface_type != 'RoofSurface':
                continue
            poslists = surface.findall(f".//{{{gml_ns}}}posList")
            for pl in poslists:
                try:
                    arr = np.array(list(map(float, pl.text.strip().split()))).reshape(-1,3)
                except Exception:
                    # try 2D fallback
                    vals = list(map(float, pl.text.strip().split()))
                    if len(vals) % 2 == 0:
                        arr = np.array(vals).reshape(-1,2)
                        # pad Z
                        arr = np.hstack([arr, np.zeros((arr.shape[0],1))])
                    else:
                        continue
                xy = arr[:, :2]
                poly = Polygon(xy)
                if not poly.is_valid:
                    poly = poly.buffer(0)
                roof_polys.append(poly)
                roof_meta.append({'building_id': b_id, 'surface_type': surface_type})
    gdf = gpd.GeoDataFrame(roof_meta, geometry=roof_polys, crs='EPSG:25832')
    return gdf

print('Parsing CityGML (this can take a while for large files)...')
gdf_gml_roofs = extract_roof_polygons_from_gml(citygml_path)
print(f'Extracted {len(gdf_gml_roofs)} roof polygons from CityGML')

Parsing CityGML (this can take a while for large files)...
Extracted 3714 roof polygons from CityGML


In [5]:
print('Loading GeoJSON roofs...')
gdf_geojson = gpd.read_file(geojson_path)
print(f'Loaded {len(gdf_geojson)} GeoJSON features')

# Normalize geometry types (ensure polygons)
gdf_geojson = gdf_geojson[gdf_geojson.geometry.type.isin(['Polygon','MultiPolygon'])].copy()


Loading GeoJSON roofs...
Loaded 7487 GeoJSON features


In [6]:
print('GML roofs CRS:', gdf_gml_roofs.crs)
print('GeoJSON CRS:', gdf_geojson.crs)

GML roofs CRS: EPSG:25832
GeoJSON CRS: EPSG:4326


In [7]:
# Reproject the GeoJSON data to match the GML data's CRS
gdf_geojson = gdf_geojson.to_crs(gdf_gml_roofs.crs) # or .to_crs('EPSG:25832')

# Check the bounds
print(f"Original GeoJSON Bounds (Lat/Lon): {gdf_geojson.total_bounds}")
#print(f"Projected GeoJSON Bounds (Meters): {gdf_geojson_projected.total_bounds}")
print(f"GML Roofs Bounds (Meters): {gdf_gml_roofs.total_bounds}")

Original GeoJSON Bounds (Lat/Lon): [ 657719.98843557 5398283.83317339  665923.8239625  5401662.37235729]
GML Roofs Bounds (Meters): [ 661995.785 5399994.03   664013.768 5401907.77 ]


In [ ]:
# ---------------------------------------------------------------------------
# STEP 6: Spatial Matching — Match each CityGML roof polygon to the GeoJSON one
# ---------------------------------------------------------------------------

print('Building spatial index and matching roofs...')
json_sindex = gdf_geojson.sindex
matches = []

for i, gml_row in gdf_gml_roofs.iterrows():
    gml_geom = gml_row.geometry
    candidate_idx = list(json_sindex.intersection(gml_geom.bounds))  # Quick bbox match
    best_j = None
    best_overlap = 0.0

    for j in candidate_idx:
        json_geom = gdf_geojson.geometry.iloc[j]
        try:
            inter = gml_geom.intersection(json_geom)
            overlap_area = inter.area
        except Exception:
            overlap_area = 0
        if overlap_area > best_overlap:
            best_overlap = overlap_area
            best_j = j

    # If no overlap, use nearest centroid distance instead
    if best_j is None or best_overlap == 0:
        gml_centroid = gml_geom.centroid
        # Use shapely geometry directly, no unsupported args
        nearest_idx = list(json_sindex.nearest(gml_centroid))
        best_j = nearest_idx[0] if nearest_idx else None
        best_dist_m = gml_centroid.distance(gdf_geojson.geometry.iloc[best_j].centroid) if best_j is not None else None
        best_overlap = 0.0
    else:
        best_dist_m = gml_geom.centroid.distance(gdf_geojson.geometry.iloc[best_j].centroid)
        
    matches.append({'gml_idx': i, 'json_idx': best_j, 'overlap_m2': best_overlap, 'centroid_dist_m': best_dist_m})
    
    

matches_df = pd.DataFrame(matches)
print('Matches computed:', len(matches_df))


Building spatial index and matching roofs...
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena
hena


In [22]:
print(len(gdf_geojson))
print(gdf_geojson.total_bounds)
print(gdf_gml_roofs.total_bounds)


7487
[11.14485741 48.7169307  11.25583649 48.74653233]
[11.20312879 48.73146581 11.23094447 48.7487482 ]


In [25]:
matches_df

,gml_idx,json_idx,overlap_m2,centroid_dist_m
0,0,2694,7.584967e-09,0.000131
1,1,2694,7.644608e-09,0.000116
2,2,2685,5.593094e-09,0.000409
3,3,2685,5.716086e-09,0.000360
4,4,2645,8.149276e-09,0.000035
...,...,...,...,...
3091,3700,3257,5.046562e-09,0.000089
3092,3703,2636,6.579223e-09,0.000110
3093,3706,2951,3.167724e-09,0.000030
3094,3707,2959,2.996921e-09,0.000005


In [28]:

# ---------------------------------------------------------------------------
# STEP 7: Merge and visualize matches
# ---------------------------------------------------------------------------
gdf_geojson = gdf_geojson.to_crs(epsg=4326)
gdf_gml_roofs = gdf_gml_roofs.to_crs(epsg=4326)
print('Building Folium visualization...')
center = [gdf_geojson.geometry.centroid.y.mean(), gdf_geojson.geometry.centroid.x.mean()]
m = folium.Map(location=center[::-1], zoom_start=12)

# Add both datasets
folium.GeoJson(gdf_geojson.to_crs(epsg=4326).to_json(), name='GeoJSON Roofs', tooltip=folium.GeoJsonTooltip(fields= [c for c in gdf_geojson.columns if c!= "geometry"])).add_to(m)
folium.GeoJson(gdf_gml_roofs.to_crs(epsg=4326).to_json(), name='GML Roofs', style_function=lambda x: {'color':'red','fill':False}).add_to(m)

# Add lines connecting matched centroids
for r in matches:
    gml_cent = gdf_gml_roofs.geometry.iloc[r['gml_idx']].centroid
    json_cent = gdf_geojson.geometry.iloc[r['json_idx']].centroid if r['json_idx'] is not None else None
    if json_cent is not None:
        folium.PolyLine(locations=[(gml_cent.y, gml_cent.x),(json_cent.y, json_cent.x)], color='green').add_to(m)

folium.LayerControl().add_to(m)
m.save('roof_matches_map.html')
print('Saved visualization map: roof_matches_map.html')

Building Folium visualization...
Saved visualization map: roof_matches_map.html


In [17]:
print(len(gdf_geojson))
print(gdf_geojson.total_bounds)
print(gdf_gml_roofs.total_bounds)


7487
[11.14485741 48.7169307  11.25583649 48.74653233]
[11.20312879 48.73146581 11.23094447 48.7487482 ]
